# 入门

## AccessKey 和 SecretKey

首先设定 KS3 的 AccessKey和SecretKey。

本例中，将它们放在环境变量配置文件 `.env` 中，名为 `KS3_ACCESS_KEY` 和 `KS3_SECRET_KEY`，然后使用 [`python-dotenv`](https://github.com/theskumar/python-dotenv) 加载

加载环境变量:

In [1]:
%load_ext dotenv

# Use find_dotenv to locate the file
%dotenv


from IPython.display import display
import os

ACCESS_KEY = os.environ['KS3_ACCESS_KEY']
SECRET_KEY = os.environ['KS3_SECRET_KEY']

## Endpoint

我自己的测试实例在上海，因此:

In [2]:
ENDPOINT = 'ks3-cn-shanghai.ksyun.com'

## 初始化客户端

In [3]:
from ks33requests.client import get_s3obj, Client

c = Client(ACCESS_KEY, SECRET_KEY, endpoint=ENDPOINT)

## 如何使用

### 列出默认项目第一个 Bucket 的 Name

发送 API 请求，并等待其返回状态码

In [4]:
res = c.send(params={'projectIds': '0'})
res

<Response [200]>

In [5]:
obj = get_s3obj(res)
obj

In [6]:
bucket_name = obj.Buckets.Bucket[0].Name
bucket_name

'huameoi-newcutter-develop'

### 获取这个BUCKET 的 Location

In [7]:
res = c.send('GET', bucket_name=bucket_name, sub_resources='location').s3obj()

In [8]:
res.valueOf_

'SHANGHAI'

### 上传一个文本文件

在内存新建一段文本，作为文件内容上传

In [9]:
import io

key = 'test'
text = '金山云对象存储服务（Kingsoft Standard Storage Service），简称KS3，是金山云为开发者提供无限制、多备份、分布式的低成本存储空间解决方案。目前提供多种语言SDK，替开发者解决存储扩容、数据可靠安全以及分布式访问等相关复杂问题，开发者可以快速的开发出涉及存储业务的程序或服务。'

stream = io.BytesIO(text.encode())

In [10]:
resp = c.send('put', bucket_name=bucket_name, object_key=key, data=stream)
resp

<Response [200]>

### 下载上一个文本文件

下载到内存，看看内容是否一致

In [11]:
resp = c.send('get', bucket_name=bucket_name, object_key=key)
resp

<Response [200]>

断言 - 字符串相同:

In [12]:
assert resp.text==text

### 删除这个文件

In [13]:
resp = c.send('delete', bucket_name=bucket_name, object_key=key)
resp

<Response [204]>

### 确定该文件不存在

In [14]:
resp = c.send('head', bucket_name=bucket_name, object_key=key, check_status=False)
resp

<Response [404]>

断言 - 404 不存在:

In [15]:
assert resp.status_code == 404